In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('../../Machine_Learning/tested.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Lets Plan

In [6]:
df.drop(columns=['PassengerId' ,'Name', 'Ticket', 'Cabin'], inplace = True)

In [7]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [8]:
# step 1 train/test/split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [10]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [11]:
y_train.sample(5)

244    0
81     0
327    0
52     1
54     0
Name: Survived, dtype: int64

## imputation transformer 

In [12]:
# we pass a list of tuples
tf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy = 'most_frequent'), [6])
], remainder= 'passthrough')

### OHE

In [13]:
tf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse=False,handle_unknown='ignore'), [1,6])
], remainder='passthrough')

## Scaling 

In [14]:
tf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
], remainder='passthrough')

## Feature Selection

In [15]:
tf4 = SelectKBest(score_func = chi2, k= 8)


## Train the model

In [16]:
tf5 = DecisionTreeClassifier()

## Creating Pipeline

In [27]:
pipe = Pipeline([
    ('tf1', tf1),
    ('tf2', tf2),
    ('tf3', tf3),
    ('tf4', tf4),
    ('tf5', tf5)
])

## Pipeline Vs make_pipeline   
Pipeline requires naming of steps, make_pipelie does not      

(Same applies to ColumnTransformer vs make_column_transformer)

In [26]:
# Alternate Syntax
# pipe = make_pipeline(tf1,tf2,tf3,tf4,tf5)

In [34]:
# pipe.fit(X_train, y_train)
pipe.named_steps

{'tf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'tf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'tf4': SelectKBest(k=8, score_func=<function chi2 at 0x00000201075B84A0>),
 'tf5': DecisionTreeClassifier()}

In [31]:

from sklearn import set_config
set_config(display='diagram')

In [36]:
# y_pred = pipe.predict(X_test)
